In [8]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import datetime

from models import DeepNetwork, DeepEnsemble
from utils import evaluate_sharpe, get_crossval_dataloaders, unload_data
from train import training, validate, train_ensembles, run_all_subsets

print(torch.cuda.is_available())

True


In [23]:
data_path = "./deep_learning/datasets/CharAll_na_rm_huge_train_variableall4_sentiment_full_new.npz"
split_lists = np.load('./deep_learning/sampling_folds/random_sampling_folds.npy', allow_pickle = True)
dataset = np.load(data_path)
data = dataset['data']

In [2]:
subset2col = {
	'flow+fund_mom+sentiment': list(range(56, 60))+[47],
	'fund_ex_mom_flow': [59]+ [x for x in range(46, 58) if x not in (list(range(54, 58))+[47])],
	'stock': range(46),
	'fund': range(46, 59),
	'fund+sentiment': range(46, 60),
	'stock+fund': range(59),
	'F_r12_2+sentiment': [58, 59],
	'stock+sentiment': [59]+list(range(0, 46)),
	'stock+fund+sentiment': range(60),
	'F_r12_2+flow+sentiment': [47, 58, 59]
}

In [4]:
config = {
	'data_path': "./deep_learning/datasets/CharAll_na_rm_huge_train_variableall4_sentiment_full_new.npz",
	'split_lists_path': './deep_learning/sampling_folds/random_sampling_folds.npy',
	'subset': 'fund+sentiment',
	'num_layers': 1,
	'hidden_dim': [2**6],
	'dropout': 0.,
	'learning_rate': 0.001,
	'epochs': 512,
	'weighted_loss': False,
	'reg_l1': 0.0,
	'reg_l2': 0.001,
	'batch_size': 300000,
	'criteria': 'Factor_sharpe',
	'ensemble_members': 8,
	'random_seed': 15,
	'device': 'cuda' if torch.cuda.is_available() else 'cpu',
	'num_workers': 4
}

config['split_lists'] = np.load(config['split_lists_path'], allow_pickle = True)
config['input_dim'] = len(subset2col[config['subset']])

In [59]:
model = DeepNetwork(config).to(device=config['device'])
print(model)

Deep_Network(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=14, out_features=64, bias=True)
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)


In [41]:
crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], 'fund+sentiment', batch_size=config['batch_size'])

for i in range(len(crossval_loaders)):
	print('Cross-validation fold {}'.format(i+1))
	for j in crossval_loaders[i]['datasets']:
		print(len(j))
	print('')

Cross-validation fold 1
208161
68181
130816

Cross-validation fold 2
198258
68181
140719

Cross-validation fold 3
208191
63344
135623



In [42]:
fold = 0
train, val, test = crossval_loaders[fold]['datasets']
train_loader, val_loader, test_loader = crossval_loaders[fold]['dataloaders']
train_mask, val_mask, test_mask = masks[fold]

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['reg_l2'])
criterion = torch.nn.MSELoss()

best_model_state = model.state_dict()
best_val_loss = torch.tensor(float('inf'))
best_val_sharpe = torch.tensor(-float('inf'))
sharpe_train = []
sharpe_val = []
loss_train = []
loss_val = []

run = 0
time_start = time.time()
for epoch in range(config['epochs']):
	epoch_loss = torch.tensor(0.0)
	y_train = []
	y_pred_train = []
	for i, (X, y) in enumerate(train_loader):
		X, y = X.float().to(config['device']), y.float().to(config['device'])
		optimizer.zero_grad()
		y_pred = model(X)
		
		loss = criterion(y_pred, y)	
		loss.backward()
		optimizer.step()	
		epoch_loss += loss.detach().cpu().item()
		
		y_train.append(y.detach().cpu())
		y_pred_train.append(y_pred.detach().cpu())
		
	epoch_loss /= i + 1
	y_train = torch.cat(y_train, dim=0)
	y_pred_train = torch.cat(y_pred_train, dim=0)
	
	sharpe_train_value = evaluate_sharpe(y_pred_train, y_train, train_mask)
	epoch_sharpe = sharpe_train_value
	sharpe_train.append(epoch_sharpe)
	loss_train.append(epoch_loss)
	
	# Validation
	with torch.no_grad():
		epoch_val_loss = torch.tensor(0.0)
		y_val = []
		y_pred_val = []
		for i, (X, y) in enumerate(val_loader):
			X, y = X.float().to(config['device']), y.float().to(config['device'])
			y_pred = model(X)
			loss = criterion(y_pred, y)
			epoch_val_loss += loss.detach().cpu().item()
			
			y_val.append(y.detach().cpu())
			y_pred_val.append(y_pred.detach().cpu())
			
	epoch_val_loss /= i + 1
	y_val = torch.cat(y_val, dim=0)
	y_pred_val = torch.cat(y_pred_val, dim=0)
	
	sharpe_val_value = evaluate_sharpe(y_pred_val, y_val, val_mask)
	epoch_val_sharpe = sharpe_val_value
	sharpe_val.append(epoch_val_sharpe)
	loss_val.append(epoch_val_loss)
	
	if epoch <= 50 or epoch % 10 == 0:
		print('Epoch {} - Training Loss: {:.8f}, Val Loss: {:.8f}, Train Sharpe: {:.8f}, Validation Sharpe: {:.8f}'.format(epoch+1, epoch_loss, epoch_val_loss, epoch_sharpe, epoch_val_sharpe))
	
	if config['criteria'] == 'Factor_sharpe':
		if epoch_val_sharpe > best_val_sharpe:
			best_val_sharpe = epoch_val_sharpe
			best_model_state = model.state_dict()
			print("Best model updated at epoch {}".format(epoch+1))
			if epoch > 50 and epoch % 10 != 0:
				print('Epoch {} - Training Loss: {:.8f}, Val Loss: {:.8f}, Train Sharpe: {:.8f}, Validation Sharpe: {:.8f}'.format(epoch+1, epoch_loss, epoch_val_loss, epoch_sharpe, epoch_val_sharpe))
	
	elif epoch_val_loss < best_val_loss:
		best_val_loss = epoch_val_loss
		best_model_state = model.state_dict()
		print("Best model updated at epoch {}".format(epoch+1))
		if epoch > 50 and epoch % 10 != 0:
			print('Epoch {} - Training Loss: {:.8f}, Val Loss: {:.8f}, Train Sharpe: {:.8f}, Validation Sharpe: {:.8f}'.format(epoch+1, epoch_loss, epoch_val_loss, epoch_sharpe, epoch_val_sharpe))

exp_path = './Experiments/'
exp_subset_path = os.path.join(exp_path, config['subset'])
if not os.path.exists(exp_subset_path):
	os.makedirs(exp_subset_path)

model.load_state_dict(best_model_state)
time_stamp = datetime.datetime.today().strftime('%Y%m%d_%H%M')
model_save_path = os.path.join(exp_subset_path, 'fold' + str(fold+1) + '_' + 'model' + str(run) + '_' + config['subset'] + '_' + time_stamp + '.pth')
torch.save(model, model_save_path)

duration = time.time() - time_start
print('Training completed in {:.0f}m {:.0f}s'.format(duration // 60, duration % 60))

In [9]:
avg_sharpe_fold_ens = 0.
for fold in range(len(crossval_loaders)):
	model_dirs = os.path.join('./Experiments/', config['subset'], 'fold' + str(fold+1))
	ensemble = DeepEnsemble(config, model_dirs)
	
	crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], config['subset'], batch_size=config['batch_size'])
	_, _, test_loader = crossval_loaders[fold]['dataloaders']
	X_test, y_test = unload_data(test_loader)
	_, _, test_mask = masks[fold]
	
	ens_pred_test = ensemble.predict(X_test.float().to(config['device']))
	sharpe_test = evaluate_sharpe(ens_pred_test, y_test, test_mask)
	print(sharpe_test)
	avg_sharpe_fold_ens += sharpe_test

avg_sharpe_fold_ens /= len(crossval_loaders)
print(avg_sharpe_fold_ens)

0.2791329795217465
0.2696935834940383
0.35179379455751314
0.300206785857766


In [10]:
config['subset'] = 'fund_ex_mom_flow'
config['input_dim'] = len(subset2col[config['subset']])

In [ ]:
crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], config['subset'], batch_size=config['batch_size'])

avg_loss_fold = {'train': 0.0, 'val': 0.0, 'test': 0.0}
avg_sharpe_fold = {'train': 0.0, 'val': 0.0, 'test': 0.0}

for fold in range(len(crossval_loaders)):
	print('\nRUNNING FOLD NO. {}'.format(fold+1))
	avg_loss_ens, avg_sharpe_ens = train_ensembles(config, crossval_loaders, masks, fold, config['ensemble_members'])
	
	avg_loss_fold['train'] += avg_loss_ens['train']
	avg_loss_fold['val'] += avg_loss_ens['val']
	avg_loss_fold['test'] += avg_loss_ens['test']
	
	avg_sharpe_fold['train'] += avg_sharpe_ens['train']
	avg_sharpe_fold['val'] += avg_sharpe_ens['val']
	avg_sharpe_fold['test'] += avg_sharpe_ens['test']

avg_loss_fold = {key: value / (len(crossval_loaders)) for key, value in avg_loss_fold.items()}
avg_sharpe_fold = {key: value / (len(crossval_loaders)) for key, value in avg_sharpe_fold.items()}

print(avg_loss_fold)
print(avg_sharpe_fold)

In [12]:
avg_sharpe_fold_ens = 0.
for fold in range(len(crossval_loaders)):
	model_dirs = os.path.join('./Experiments/', config['subset'], 'fold' + str(fold+1))
	ensemble = DeepEnsemble(config, model_dirs)
	
	crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], config['subset'], batch_size=config['batch_size'])
	_, _, test_loader = crossval_loaders[fold]['dataloaders']
	X_test, y_test = unload_data(test_loader)
	_, _, test_mask = masks[fold]
	
	ens_pred_test = ensemble.predict(X_test.float().to(config['device']))
	sharpe_test = evaluate_sharpe(ens_pred_test, y_test, test_mask)
	print(sharpe_test)
	avg_sharpe_fold_ens += sharpe_test

avg_sharpe_fold_ens /= len(crossval_loaders)
print(avg_sharpe_fold_ens)

0.00015002249363720127
0.10384931098224807
0.06541024123059637
0.056469858235493886


In [13]:
config['subset'] = 'stock+fund'
config['input_dim'] = len(subset2col[config['subset']])

In [ ]:
crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], config['subset'], batch_size=config['batch_size'])

avg_loss_fold = {'train': 0.0, 'val': 0.0, 'test': 0.0}
avg_sharpe_fold = {'train': 0.0, 'val': 0.0, 'test': 0.0}

for fold in range(len(crossval_loaders)):
	print('\nRUNNING FOLD NO. {}'.format(fold+1))
	avg_loss_ens, avg_sharpe_ens = train_ensembles(config, crossval_loaders, masks, fold, config['ensemble_members'])
	
	avg_loss_fold['train'] += avg_loss_ens['train']
	avg_loss_fold['val'] += avg_loss_ens['val']
	avg_loss_fold['test'] += avg_loss_ens['test']
	
	avg_sharpe_fold['train'] += avg_sharpe_ens['train']
	avg_sharpe_fold['val'] += avg_sharpe_ens['val']
	avg_sharpe_fold['test'] += avg_sharpe_ens['test']

avg_loss_fold = {key: value / (len(crossval_loaders)) for key, value in avg_loss_fold.items()}
avg_sharpe_fold = {key: value / (len(crossval_loaders)) for key, value in avg_sharpe_fold.items()}

print(avg_loss_fold)
print(avg_sharpe_fold)

In [16]:
avg_sharpe_fold_ens = 0.
for fold in range(len(crossval_loaders)):
	model_dirs = os.path.join('./Experiments/', config['subset'], 'fold' + str(fold+1))
	ensemble = DeepEnsemble(config, model_dirs)
	
	crossval_loaders, masks = get_crossval_dataloaders(config['data_path'], config['split_lists'], config['subset'], batch_size=config['batch_size'])
	_, _, test_loader = crossval_loaders[fold]['dataloaders']
	X_test, y_test = unload_data(test_loader)
	_, _, test_mask = masks[fold]
	
	ens_pred_test = ensemble.predict(X_test.float().to(config['device']))
	sharpe_test = evaluate_sharpe(ens_pred_test, y_test, test_mask)
	print(sharpe_test)
	avg_sharpe_fold_ens += sharpe_test

avg_sharpe_fold_ens /= len(crossval_loaders)
print(avg_sharpe_fold_ens)

0.09801895666399872
0.1920559573122067
0.14477015631698126
0.14494835676439558
